#H2O AutoML부터 수행

In [ ]:
# 종속 패키지 설차
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
# 기존 h2o 삭제
!pip uninstall h2o
# h2o 설치
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

!pip install autogluon

In [ ]:
#기본 라이브러리
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time

#H2O 라이브러리
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
h2o.init()
h2o.no_progress()

#열과행 많이보이고 오류 안뜨도록
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
##데이터 불러오기

#  A-2월로 A월 예측
df2_train = pd.read_csv('/content/drive/Shareddrives/알빠진 포도/데이터 모음집/df2_train_최종_13일클러변경.csv', encoding='cp949', index_col='emd_nm_2term')
# 데이터 확인

#트레인셋 분리
# - 클러스터링별 택1
#df2_train = df2_train[df2_train['clustered']==1]        #클러1
df2_train = df2_train[df2_train['clustered']==2]        #클러2
#df2_train = df2_train[df2_train['clustered']==3]        #클러3

df2_train = df2_train.drop(['Unnamed: 0','emd_nm','clustered'], axis=1)

'''
#스케일링 필요시 하기
#df2_train = df2_train.fillna(0.5)  #필요시 사용
min_max_scaler = MinMaxScaler()
fitted = min_max_scaler.fit(df2_train)
print(fitted.data_max_)
data_scale = min_max_scaler.transform(df2_train)
df2_train = pd.DataFrame(data_scale, columns=df2_train.columns, index=df2_train.index)
'''

#결측치 있는지 확인.
df2_train.isnull().sum()

In [ ]:
#  test 가져오기
df2_test = pd.read_csv('/content/drive/Shareddrives/알빠진 포도/데이터 모음집/df2_test_최종_13일클러변경.csv', encoding='cp949', index_col='emd_nm_2term')

# - 클러스터링별 택1
#df2_test = df2_test[df2_test['clustered']==1]        #클러1
df2_test = df2_test[df2_test['clustered']==2]        #클러2
#df2_test = df2_test[df2_test['clustered']==3]        #클러3

#불필요한 칼럼 제거
df2_test = df2_test.drop(['Unnamed: 0','emd_nm','clustered'], axis=1)

'''
#스케일링 필요시 하기
#df2_test = df2_test.fillna(0.5)  #필요시 사용
min_max_scaler = MinMaxScaler()
fitted = min_max_scaler.fit(df2_test)
print(fitted.data_max_)
data_scale = min_max_scaler.transform(df2_test)
df2_test = pd.DataFrame(data_scale, columns=df2_test.columns, index=df2_test.index)
'''

#결측치 있는지 확인. 결측치가 차후 오류일으킬수 있기때문에 채우든 삭제하든 해야함
df2_test.isnull().sum()

In [ ]:
## y에 타겟값 두고 데이터 분리
y = "em_g_interpolated"
x = list(df2_train.columns)  
x.remove(y)  #테스트셋에서 y값에 넣은 칼럼은 제거

# 검증셋 분리
train, valid = train_test_split(df2_train,
                                test_size=0.2, 
                                shuffle=True)
h2o_train = h2o.H2OFrame(train, )
h2o_valid = h2o.H2OFrame(valid)

In [ ]:
'''
# 분류인경우에는 아래 사용
h2o_train[y] = h2o_train[y].asfactor()
h2o_valid[y] = h2o_valid[y].asfactor()

def _convert_h2oframe_to_numeric(h2o_frame, training_columns):
    for column in training_columns:
        h2o_frame[column] = h2o_frame[column].asnumeric()
    return h2o_frame

_convert_h2oframe_to_numeric(h2o_train, df2_train.columns)
_convert_h2oframe_to_numeric(h2o_valid, df2_train.columns)
'''

'\n#혹시 쓸지몰라서 남겨둠.\n\ndef _convert_h2oframe_to_numeric(h2o_frame, training_columns):\n    for column in training_columns:\n        h2o_frame[column] = h2o_frame[column].asnumeric()\n    return h2o_frame\n\n#_convert_h2oframe_to_numeric(h2o_train, df2_train.columns)\n#_convert_h2oframe_to_numeric(h2o_valid, df2_train.columns)\n'

In [ ]:
## 학습시간설정
max_runtime_secs = 600

#학습시작
aml = H2OAutoML(max_runtime_secs=max_runtime_secs)
aml.train(x = x, y = y, training_frame=h2o_train, leaderboard_frame=h2o_valid)  #leaderboard

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_4_AutoML_5_20210914_212821

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 4343954868520.31
RMSE: 2084215.6482764229
MAE: 1626545.0667869586
RMSLE: 0.045479034114908334
R^2: 0.9991981221729346
Mean Residual Deviance: 4343954868520.31
Null degrees of freedom: 447
Residual degrees of freedom: 444
Null deviance: 2.4269180608460524e+18
Residual deviance: 1946091781097099.0
AIC: 14318.082152238592

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 1425189956632302.0
RMSE: 37751688.129569806
MAE: 7222414.831959845
RMSLE: NaN
R^2: 0.7369152626649913
Mean Residual Deviance: 1425189956632302.0
Null degrees of freedom: 447
Residual degrees of freedom: 444
Null deviance: 2.436922889981055e+18
Residual deviance: 6.384851005712713e+17
AIC: 16913.469438937296


In [ ]:
## 성능평가

# Print Leaderboard (ranked by xval metrics)
leaderboard = aml.leaderboard
performance = aml.leader.model_performance(h2o_valid)  # 최상결과 보여준거 따로 저장
model_id   =aml.leader.model_id  # 최고 모델 명
variable_importance=aml.leader.varimp()  # 중요한 입력 변수

'''
#분류모델의 경우애는 아래 사용
accuracy   =performance.accuracy()  # 정확도
precision  =performance.precision()  # precision
recall     =performance.recall()  # recall
F1         =performance.F1()  # f1
auc        =performance.auc()  # auc
'''

#성능나온거 출력
print(model_id)
print()
print(variable_importance)
print()
print(performance)

## 중요 변수 시각화. 변수중요도가 뽑히지 않는 모델도 있음
try :
  aml.leader.varimp_plot()
except:
  print('변수중요도 출력불가')

## automl 결과 확인
get_aml = h2o.automl.get_automl(aml.project_name)
label_predicted = get_aml.predict(h2o_valid)
print(label_predicted['predict'])
print(h2o_valid['em_g'])


StackedEnsemble_BestOfFamily_4_AutoML_4_20210914_211850

None


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 46321083924553.18
RMSE: 6805959.441882767
MAE: 4833925.477819918
RMSLE: 0.08536443913637193
R^2: 0.9930092554734873
Mean Residual Deviance: 46321083924553.18
Null degrees of freedom: 111
Residual degrees of freedom: 108
Null deviance: 7.53492831231955e+17
Residual deviance: 5187961399549956.0
AIC: 3852.1034866052146



In [ ]:
#테스트셋 넣어서 결과보기
y = "em_g_interpolated"
x = list(df2_test.columns) 
x.remove(y)

valid = df2_test
h2o_valid = h2o.H2OFrame(valid)      
label_predicted = get_aml.predict(h2o_valid)
print(label_predicted['predict'])
print(h2o_valid['em_g'])
df2_pred_list = h2o.as_list(label_predicted['predict'], use_pandas=False)

df2_pred_list

# autogluon

In [ ]:
from autogluon.tabular import TabularPredictor as task
#from autogluon import TabulerPrediction as task #AutoGluon의 로딩

predictor = task(label='em_g').fit(train_data = df2_train ) #학습
predictions = predictor.predict(df2_test) #테스트 데이터에 대한 예측
predictions

No path specified. Models will be saved in: "AutogluonModels/ag-20210910_121606/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20210910_121606/"
AutoGluon Version:  0.3.1
Train Data Rows:    976
Train Data Columns: 102
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1.0, 0.0, 0.22682, 0.2099)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11790.07 MB
	Train Data (Original)  Memory Usage: 0.8 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify speci

[1000]	train_set's rmse: 0.00116268	valid_set's rmse: 0.0199576
[2000]	train_set's rmse: 0.00017399	valid_set's rmse: 0.0198837
[3000]	train_set's rmse: 3.40856e-05	valid_set's rmse: 0.0198763
[4000]	train_set's rmse: 7.0074e-06	valid_set's rmse: 0.0198754


	-0.0199	 = Validation score   (root_mean_squared_error)
	13.44s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: LightGBM ...
	-0.0264	 = Validation score   (root_mean_squared_error)
	3.74s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.032	 = Validation score   (root_mean_squared_error)
	7.6s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	-0.0192	 = Validation score   (root_mean_squared_error)
	69.72s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-0.0291	 = Validation score   (root_mean_squared_error)
	2.47s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-0.027	 = Validation score   (root_mean_squared_error)
	6.42s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: XGBoost ...
	-0.0382	 = Validation score   (root_mean_squared_error)
	7.04s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model

[1000]	train_set's rmse: 0.000197496	valid_set's rmse: 0.0266327
[2000]	train_set's rmse: 9.08731e-06	valid_set's rmse: 0.0266088
[3000]	train_set's rmse: 3.99445e-07	valid_set's rmse: 0.0266077
[4000]	train_set's rmse: 1.78548e-08	valid_set's rmse: 0.0266076
[5000]	train_set's rmse: 7.99111e-10	valid_set's rmse: 0.0266076
[6000]	train_set's rmse: 3.36798e-11	valid_set's rmse: 0.0266076
[7000]	train_set's rmse: 1.4545e-12	valid_set's rmse: 0.0266076
[8000]	train_set's rmse: 6.3694e-14	valid_set's rmse: 0.0266076
[9000]	train_set's rmse: 2.90363e-15	valid_set's rmse: 0.0266076
[10000]	train_set's rmse: 8.9544e-16	valid_set's rmse: 0.0266076


	-0.0266	 = Validation score   (root_mean_squared_error)
	276.41s	 = Training   runtime
	0.73s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-0.0189	 = Validation score   (root_mean_squared_error)
	0.32s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 399.02s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20210910_121606/")


emd_nm_2term
건입동43     0.113741
건입동44     0.088559
남원읍43     0.174292
남원읍44     0.150987
노형동43     0.788926
노형동44     0.710613
대륜동43     0.163957
대륜동44     0.116626
대정읍43     0.342247
대정읍44     0.301706
대천동43     0.179773
대천동44     0.134113
도두동43     0.042002
도두동44     0.036219
봉개동43     0.030207
봉개동44     0.019985
삼도1동43    0.197668
삼도1동44    0.177912
삼양동43     0.325129
삼양동44     0.288579
송산동43     0.057019
송산동44     0.061465
아라동43     0.494882
아라동44     0.413519
연동43      0.627389
연동44      0.549016
예래동43     0.049344
예래동44     0.028649
오라동43     0.188543
오라동44     0.166599
외도동43     0.307713
외도동44     0.283654
용담1동43    0.085470
용담1동44    0.074081
용담2동43    0.184355
용담2동44    0.159823
이도2동43    0.727064
이도2동44    0.667773
이호동43     0.036442
이호동44     0.031684
일도2동43    0.471917
일도2동44    0.420439
정방동43     0.051047
정방동44     0.047533
중문동43     0.183404
중문동44     0.154185
천지동43     0.063089
천지동44     0.072372
화북동43     0.349054
화북동44     0.309035
Name: em_g, dtype: float32

# 두가지 AutoML에서 나온 결과값 비교

In [ ]:
#데이터프레임으로 만들기
df2_dic ={'h2o':df2_pred_list[1:] ,
          'gluon': predictions.values.tolist()}
df2_pred = pd.DataFrame(df2_dic)

#저장 및 출력
df2_pred.to_csv('/content/drive/MyDrive/autoML/df2_pred3-5분-스케.csv', encoding='cp949')
df2_pred

,h2o,gluon
0,[0.11851364022680977],0.113741
1,[0.0905331868699123],0.088559
2,[0.162325894610537],0.174292
3,[0.15584735028888894],0.150987
4,[0.8323401209623568],0.788926
5,[0.8027571179691264],0.710613
6,[0.15762448178507918],0.163957
7,[0.13668713345595798],0.116626
8,[0.37086123057360343],0.342247
9,[0.33455825380115894],0.301706
